In [47]:
import numpy as np
import torch
from torchvision.utils import save_image

In [48]:
%run loader.ipynb

In [49]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [50]:
def test(epoch, model, loader):
    model.eval()
    test_losses = []
    with torch.no_grad():
        for i, data in enumerate(loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = ((inputs - outputs) ** 2).sum() + model.encoder.kl# + (inputs - sr).abs().sum()
            # l = loss(inputs, outputs, mean, logvar).to(device)
            test_losses.append(loss.item() / len(labels))
            if i == 0:
                n = min(inputs.size(0), 8)
                comparison = torch.cat([inputs[:n],
                                        outputs.view(len(labels), 1, 100, 100)[:n]])
                save_image(comparison.cpu(),
                           '../results/reconstruction/' + str(epoch) + '.png', nrow=n)
    return np.mean(test_losses)
def loss(X, X_hat, mean, logvar):
    reconstruction_loss = BCE_loss(X_hat, X)
    KL_divergence = 0.5 * torch.sum(-1 - logvar + torch.exp(logvar) + mean ** 2)
    return reconstruction_loss + KL_divergence